In [2]:
pip install fasttext

  Using cached pybind11-2.6.2-py2.py3-none-any.whl (191 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-win_amd64.whl size=235144 sha256=b171b48fc736fa29f7ad3ca11f98c0471648cfe5baaa58d212b33385345a27e7
  Stored in directory: c:\users\glase\appdata\local\pip\cache\wheels\4e\ca\bf\b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext
Note: you may need to restart the kernel to use updated packages.


In [68]:
import fasttext
import pandas as pd
import nltk
import emoji
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score

In [7]:
data = pd.read_csv("Danish/Danish/offenseval-da-training-v1.tsv", sep='\t')[:-1]

texts = data["tweet"].apply(str)
labels = data["subtask_a"].apply(str)
#print(data.loc[2883])
texts

0       Jeg tror det vil være dejlig køligt, men jeg v...
1       Så kommer de nok til at investere i en ny cyke...
2       Nu er det jo også de Ikea-aber der har lavet s...
3       128 Varme emails, er vi enige om at det er sex...
4       Desværre tyder det på, at amerikanerne er helt...
                              ...                        
2955    Har sgu lidt en anelse om... det her kunne mås...
2956    Ind og ruske tremmer med hende,Den syge kælling!!
2957                                             fedtmule
2958                                    ##HAR I HØRT DET?
2959    Kommer det bag på nogen? Det er jo fucking var...
Name: tweet, Length: 2960, dtype: object

In [8]:
#tokenization
texts_tokenized = []
for el in texts:
    texts_tokenized.append([emoji.demojize(el) for el in el.split()]) #nltk.word_tokenize(el)

    
texts_tokenized[2883]

['@USER',
 'næste',
 'gang',
 'pastaen',
 'er',
 'brændt',
 'på',
 '!',
 ':face_with_tears_of_joy:',
 'det',
 'jo',
 'sygt...']

In [26]:
#normalization
texts_normalized = []
for sent in texts_tokenized:
    # remove emojis
    sent = [el.lower() for el in sent if not bool(re.search(r":.*:", el))] #or replace with just name?
    # remove repeating characters
    sent = [re.sub(r"(.)\1{2,}", r"\1", el) for el in sent]
    sent = [re.sub(r"[!?\(\)\\&.,:><_#\[\]/]+", "", el)for el in sent]
    texts_normalized.append([el for el in sent if el != ""])
        
print(texts_normalized[2883])

['@user', 'næste', 'gang', 'pastaen', 'er', 'brændt', 'på', 'det', 'jo', 'sygt']


In [27]:
texts_normalized2 = [" ".join(el) for el in texts_normalized]
texts_normalized3 = pd.DataFrame(texts_normalized2, columns=["tweet"])
texts_normalized3
data["tweet"] = texts_normalized2
data

,id,tweet,subtask_a
0,3131,jeg tror det vil være dejlig køligt men jeg vi...,NOT
1,711,så kommer de nok til at investere i en ny cyke...,NOT
2,2500,nu er det jo også de ikea-aber der har lavet s...,OFF
3,2678,128 varme emails er vi enige om at det er sext...,NOT
4,784,desværre tyder det på at amerikanerne er helt ...,NOT
...,...,...,...
2955,170,har sgu lidt en anelse om det her kunne måske ...,NOT
2956,1226,ind og ruske tremmer med hendeden syge kælling,OFF
2957,2596,fedtmule,NOT
2958,1802,har i hørt det,NOT


In [28]:
texts = data["tweet"].apply(str)
texts, labels

(0       jeg tror det vil være dejlig køligt men jeg vi...
 1       så kommer de nok til at investere i en ny cyke...
 2       nu er det jo også de ikea-aber der har lavet s...
 3       128 varme emails er vi enige om at det er sext...
 4       desværre tyder det på at amerikanerne er helt ...
                               ...                        
 2955    har sgu lidt en anelse om det her kunne måske ...
 2956       ind og ruske tremmer med hendeden syge kælling
 2957                                             fedtmule
 2958                                       har i hørt det
 2959    kommer det bag på nogen det er jo fucking varm...
 Name: tweet, Length: 2960, dtype: object,
 0       NOT
 1       NOT
 2       OFF
 3       NOT
 4       NOT
        ... 
 2955    NOT
 2956    OFF
 2957    NOT
 2958    NOT
 2959    OFF
 Name: subtask_a, Length: 2960, dtype: object)

In [29]:
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.25, random_state=123)
texts_train, texts_test, labels_train, labels_test

(2734    jeg vil foreslå to andre billeder af københavn...
 2886    og der bliver stadigt flereurl danmark længe leve
 2738                                                  lol
 2937    ah og vi har allerede fået bygget en mur mod i...
 2671    vi må da sige han går da all in når man skal t...
                               ...                        
 1147    det er jo ikke sjovt nogen ting spøger man bar...
 2154                                  det har du os rat i
 1766                          grønt er altså for bornholm
 1122    de skal bare kvæles langsomt så de ligger stil...
 1346    syntes man begynder at høre mere om partering ...
 Name: tweet, Length: 2220, dtype: object,
 418     ikke noget andre kvinder ikke har men ser bedr...
 1224    foretrækker dog isterningebakken den kan bruge...
 1617    troede at vores flag altid brandte det gør det...
 1745                                                 mere
 2139                        også kendt som opium med brus
             

In [30]:
train_file = "Danish/Model/train.txt"
test_file = "Danish/Model/test.txt"
with open(train_file, "w", encoding="utf8") as f:
    for label, tweet in zip(labels_train, texts_train):
        f.write(f"__label__{label} {tweet}\n")
        
with open(test_file, "w", encoding="utf8") as f:
    for label, tweet in zip(labels_test, texts_test):
        f.write(f"__label__{label} {tweet}\n")

In [31]:
model = fasttext.train_supervised(input=train_file)

In [38]:
print(model.predict("ikke noget andre kvinder ikke har men ser bedre ud med tøj på"),
model.predict("de skulle have en røvfuld så stor at de ikke ved hvad de selv hedder og så skal de eller deres forældre selv betale regningen"),
model.test(test_file))

(('__label__NOT',), array([0.95414782])) (('__label__NOT',), array([0.89170223])) (740, 0.9135135135135135, 0.9135135135135135)


In [39]:
model = fasttext.train_supervised(input=train_file, epoch=25)

In [40]:
print(model.predict("ikke noget andre kvinder ikke har men ser bedre ud med tøj på"),
model.predict("de skulle have en røvfuld så stor at de ikke ved hvad de selv hedder og så skal de eller deres forældre selv betale regningen"),
model.test(test_file))

(('__label__NOT',), array([0.99845523])) (('__label__NOT',), array([0.98049331])) (740, 0.9256756756756757, 0.9256756756756757)


In [42]:
model = fasttext.train_supervised(input=train_file, epoch=25, lr=0.3)

In [43]:
print(model.predict("ikke noget andre kvinder ikke har men ser bedre ud med tøj på"),
model.predict("de skulle have en røvfuld så stor at de ikke ved hvad de selv hedder og så skal de eller deres forældre selv betale regningen"),
model.test(test_file))

(('__label__NOT',), array([0.99983138])) (('__label__NOT',), array([0.99922681])) (740, 0.9243243243243243, 0.9243243243243243)


In [90]:
model = fasttext.train_supervised(input=train_file, epoch=1000, lr=0.1)

In [91]:
print(model.predict("ikke noget andre kvinder ikke har men ser bedre ud med tøj på"),
model.predict("hvad med i stedet for at anholde ham så give ham et lift til hotellet i stedet for de er skøre de svenskere"),
model.test(test_file))

(('__label__NOT',), array([1.00000584])) (('__label__OFF',), array([0.99513948])) (740, 0.9216216216216216, 0.9216216216216216)


In [92]:
#model.predict("hvad med i stedet for at anholde ham så give ham et lift til hotellet i stedet for de er skøre de svenskere")[0][0][-3:]
tp = 0
f = 0
all = 0
pred = [model.predict(t)[0][0][-3:] for t in texts_test]
pn = []
for el in pred:
    if el == "OFF":
        pn.append(1)
    else:
        pn.append(0)

tn = []
for el in texts_test:
    if el == "OFF":
        tn.append(1)
    else:
        tn.append(0)

accuracy_score(labels_test, pred), f1_score(tn, pn)

(0.9216216216216216, 0.0)

In [94]:
tp = 0
f = 0
all = 0
for t, l in zip(texts_test, labels_test):
    pred_label = model.predict(t)[0][0][-3:]
    if pred_label == l:
        tp += 1
        all += 1
    else:
        f += 1
        all += 1
print(tp, f, all)

682 58 740
